# Does the Rowmodctr Update for Non-Updating Updates?

* Source: https://www.brentozar.com/archive/2019/05/does-the-rowmodctr-update-for-non-updating-updates/
* Last updated: 2019/05/16
* License: [CC BY-SA 3.0: Creative Commons Attribution-ShareAlike 3.0 Unported](https://creativecommons.org/licenses/by-sa/3.0/)
* Data contents: made-up data
* Requires: any supported version of SQL Server or Azure SQL DB

Okay, look, it's a mouthful of a blog post title, and there are only gonna be maybe six of us in the world who get excited enough to check this kind of thing, but if you're in that intimate group, then the title's already got you interested in the demo. (Shout out to Riddhi P. for asking this cool question in class.)

The system table sys.sysindexes has a rowmodctr that tells you how many times rows have been updated in an object. This comes in handy when we need to figure out if it's time to update the statistics on a table.

To set the stage, let's create a table, put a couple of indexes on it, and check rowmodctr to see how many changes have been made:

In [6]:
/* Create a test table with two indexes */
CREATE TABLE dbo.People (ID INT IDENTITY(1,1), PersonName VARCHAR(200));
CREATE UNIQUE CLUSTERED INDEX CL_ID ON dbo.People(ID);
CREATE NONCLUSTERED INDEX IX_PersonName ON dbo.People(PersonName);

/* Check the rowmodctr: */
SELECT name, rows, rowcnt, rowmodctr
FROM sys.sysindexes i
WHERE name IN ('CL_ID', 'IX_PersonName');

(2 rows affected)

Total execution time: 00:00:01.336

name,rows,rowcnt,rowmodctr
CL_ID,0,0,0
IX_PersonName,0,0,0


No rows have been modified yet since no data has been loaded. Let's load 1,000 rows of data, and then check rowmodctr to see how many modifications have been made to the table:

In [7]:
/* Insert 1,000 rows. */
INSERT INTO dbo.People(PersonName)
SELECT TOP 1000 'Slim Shady'
FROM sys.all_columns;

/* Check the rowmodctr: */
SELECT name, rows, rowcnt, rowmodctr
FROM sys.sysindexes i
WHERE name IN ('CL_ID', 'IX_PersonName');

(1000 rows affected)

(2 rows affected)

Total execution time: 00:00:06.550

name,rows,rowcnt,rowmodctr
CL_ID,1000,1000,1000
IX_PersonName,1000,1000,1000


Rowmodctr is 1,000 because 1,000 rows have been modified in the table - hey, inserts count as modifications.

Now the fun starts: let's update everybody's PersonName to be the same thing:

In [8]:
/* Update the name to be its own value again: */
UPDATE dbo.People
SET PersonName = 'Slim Shady';
GO

/* Check rowmodctr again: */
SELECT name, rows, rowcnt, rowmodctr
FROM sys.sysindexes i
WHERE name IN ('CL_ID', 'IX_PersonName');

(1000 rows affected)

(2 rows affected)

Total execution time: 00:00:01.709

name,rows,rowcnt,rowmodctr
CL_ID,1000,1000,1000
IX_PersonName,1000,1000,2000


Remember, we had rowmodctr 1,000 for both indexes just a second ago.

I'm gonna be honest with you, dear reader: this was not the result I expected.

As a naive, delicate flower, I expected SQL Server to treat the rowmodctr the same way on both the clustered and nonclustered indexes. Instead, here we're showing zero new modifications on the clustered index, but 1,000 new modifications on the name index.

In the big scheme of things, this isn't a showstopper problem. It's not like I'm over here waving my hands going, "OMG, this violates the laws of space and time!" It's a system table, and if anybody's relying on this thing for razor-sharp accuracy, they probably deserve what's coming to them. I just find it so amusing that it's handled differently on the two kinds of indexes - even though neither of them were modified. Huh.

How about if we explicitly tell SQL Server to set it to the same value:

In [9]:
UPDATE dbo.People
SET PersonName = PersonName;

/* Check the rowmodctr: */
SELECT name, rows, rowcnt, rowmodctr
FROM sys.sysindexes i
WHERE name IN ('CL_ID', 'IX_PersonName');

(1000 rows affected)

(2 rows affected)

Total execution time: 00:00:01.834

name,rows,rowcnt,rowmodctr
CL_ID,1000,1000,1000
IX_PersonName,1000,1000,3000


The nonclustered index shows modifications, and the clustered index doesn't.

## So what's the takeaway?

I don't think this is a performance-shattering problem: it's probably fairly rare that you find an application that runs updates even when no changes were made.

To be clear, the fix isn't to switch to a delete-the-old-row-and-insert-a-new-row design pattern: that would be twice as bad, since now you're doubling the number of rows modified with each operation.

In [10]:
DROP TABLE dbo.People;

Commands completed successfully.

Total execution time: 00:00:00.891